# Building an Agent
Based on: https://python.langchain.com/docs/tutorials/agents/ 

## Setup
Get the API keys for LangChain, TAVILY (search engine) and your API of choice (ChatNVIDIA, in this case)

In [2]:
# %pip install -U langchain-community langgraph langchain-anthropic tavily-python langgraph-checkpoint-sqlite
# !pip install -qU langchain-openai
!pip install -qU langchain-nvidia-ai-endpoints

In [3]:
!export LANGCHAIN_TRACING_V2="true"
!export LANGCHAIN_API_KEY="..."

In [4]:
import getpass
import os

os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass()

 ········


In [5]:
import getpass
import os

os.environ["TAVILY_API_KEY"] = getpass.getpass()

 ········


In [6]:
from langchain_community.tools.tavily_search import TavilySearchResults

search = TavilySearchResults(max_results=2)
search_results = search.invoke("what is the weather in SF")
print(search_results)
# If we want, we can create other tools.
# Once we have all the tools we want, we can put them in a list that we will reference later.
tools = [search]

[{'url': 'https://www.weatherapi.com/', 'content': "{'location': {'name': 'San Francisco', 'region': 'California', 'country': 'United States of America', 'lat': 37.775, 'lon': -122.4183, 'tz_id': 'America/Los_Angeles', 'localtime_epoch': 1734542830, 'localtime': '2024-12-18 09:27'}, 'current': {'last_updated_epoch': 1734542100, 'last_updated': '2024-12-18 09:15', 'temp_c': 7.4, 'temp_f': 45.4, 'is_day': 1, 'condition': {'text': 'Fog', 'icon': '//cdn.weatherapi.com/weather/64x64/day/248.png', 'code': 1135}, 'wind_mph': 5.4, 'wind_kph': 8.6, 'wind_degree': 40, 'wind_dir': 'NE', 'pressure_mb': 1027.0, 'pressure_in': 30.33, 'precip_mm': 0.0, 'precip_in': 0.0, 'humidity': 98, 'cloud': 100, 'feelslike_c': 6.0, 'feelslike_f': 42.8, 'windchill_c': 6.0, 'windchill_f': 42.8, 'heatindex_c': 7.4, 'heatindex_f': 45.4, 'dewpoint_c': 6.9, 'dewpoint_f': 44.4, 'vis_km': 10.0, 'vis_miles': 6.0, 'uv': 0.5, 'gust_mph': 9.8, 'gust_kph': 15.7}}"}, {'url': 'https://www.msn.com/en-us/weather/topstories/decemb

## Selecting and LLM
We will use NVIDIA models to build our Agent.

In [8]:
import getpass
import os

if not os.environ.get("NVIDIA_API_KEY"):
  os.environ["NVIDIA_API_KEY"] = getpass.getpass("Enter API key for NVIDIA: ")

from langchain_nvidia_ai_endpoints import ChatNVIDIA

model = ChatNVIDIA(model="meta/llama-3.1-70b-instruct")

Enter API key for NVIDIA:  ········


In [9]:
from langchain_core.messages import HumanMessage

response = model.invoke([HumanMessage(content="hi!")])
response.content

'Hello! How can I assist you today?'

In [10]:
# Get a list of models that support tool calling
models_list = ChatNVIDIA.get_available_models()
for m in models_list:
    if 'supports_tools=True' in str(m):
        print(m)

id='mistralai/mistral-large-2-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True base_model=None
id='meta/llama-3.1-405b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True base_model=None
id='meta/llama-3.2-3b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True base_model=None
id='meta/llama-3.1-70b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True base_model=None
id='nv-mistralai/mistral-nemo-12b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True base_model=None
id='meta/llama-3.1-8b-instruct' model_type='chat' client='ChatNVIDIA' endpoint=None aliases=None supports_tools=True supports_structured_output=True bas

In [11]:
model_with_tools = model.bind_tools(tools)

In [12]:
response = model_with_tools.invoke([HumanMessage(content="Hi!")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: It's nice to meet you. Is there something I can help you with or would you like to chat?
ToolCalls: []


The following is not calling the tool (because we have not created the agent. The LLM recognises that the tool needs to be called

In [14]:
response = model_with_tools.invoke([HumanMessage(content="What's the weather in SF?")])

print(f"ContentString: {response.content}")
print(f"ToolCalls: {response.tool_calls}")

ContentString: 
ToolCalls: [{'name': 'tavily_search_results_json', 'args': {'query': 'San Francisco weather'}, 'id': 'chatcmpl-tool-1f5a1fd778fe49169668e9a35b20cb0b', 'type': 'tool_call'}]


## Agent Creation

We will use `model` as opposed to `model_with_tools` as the binding will happen underneath

In [16]:
from langgraph.prebuilt import create_react_agent

agent_executor = create_react_agent(model, tools)

In [17]:
# response = agent_executor.invoke({"messages": [HumanMessage(content="hi!")]})

# response["messages"]

In [18]:
# response = agent_executor.invoke(
#     {"messages": [HumanMessage(content="whats the weather in sf?")]}
# )
# response["messages"]

## Memory makes the agent useful

In [20]:
from langgraph.checkpoint.memory import MemorySaver

memory = MemorySaver()

In [21]:
agent_executor = create_react_agent(model, tools, checkpointer=memory)

config = {"configurable": {"thread_id": "abc123"}}

In [22]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="hi im bob!")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content="Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", additional_kwargs={}, response_metadata={'role': 'assistant', 'content': "Hi Bob! It's nice to meet you. Is there something I can help you with or would you like to chat?", 'token_usage': {'prompt_tokens': 317, 'total_tokens': 342, 'completion_tokens': 25}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-70b-instruct'}, id='run-48b96f67-b0d7-49fe-abb1-f80c6822cf86-0', usage_metadata={'input_tokens': 317, 'output_tokens': 25, 'total_tokens': 342}, role='assistant')]}}
----


In [23]:
for chunk in agent_executor.stream(
    {"messages": [HumanMessage(content="whats my name?")]}, config
):
    print(chunk)
    print("----")

{'agent': {'messages': [AIMessage(content='Your name is Bob.', additional_kwargs={}, response_metadata={'role': 'assistant', 'content': 'Your name is Bob.', 'token_usage': {'prompt_tokens': 357, 'total_tokens': 362, 'completion_tokens': 5}, 'finish_reason': 'stop', 'model_name': 'meta/llama-3.1-70b-instruct'}, id='run-b8f6a260-6e53-4aad-be0e-839eee401160-0', usage_metadata={'input_tokens': 357, 'output_tokens': 5, 'total_tokens': 362}, role='assistant')]}}
----
